In [121]:
#Vamos a calcular como necesitamos las métricas para realizar el modelo NaiveBayes

import csv 
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import confusion_matrix, recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [122]:
#Leemos los csv de las tablas 
nodes = pd.read_csv('../Tablas/TablaAtributos.csv')
nodesNoCentrality = pd.read_csv('../Tablas/TablaNoCentrality.csv')
nodesNoClustering = pd.read_csv('../Tablas/TablaNoClustering.csv')
nodesNoCommunities = pd.read_csv('../Tablas/TablaNoCommunities.csv')
nodesNoKCore = pd.read_csv('../Tablas/TablaNoKCore.csv')


In [123]:
ad=['name']
ac=['Closeness_Centrality','Betweenness_Centrality','Degree_Centrality','Clustering_Coefficient','Triangles','Squares','K_Core', 'Comunidad','asyn_lpa_community']
atributtes = nodes.loc[:, ['id']+ad+ac]

ad_NoCentrality=['name']
ac_NoCentrality=['Clustering_Coefficient','Triangles','Squares','K_Core', 'Comunidad','asyn_lpa_community']
atributtesNoCentrality = nodesNoCentrality.loc[:, ['id']+ad_NoCentrality+ac_NoCentrality]

ad_NoClustering=['name']
ac_NoClustering=['Closeness_Centrality','Betweenness_Centrality','Degree_Centrality','Triangles','Squares','K_Core', 'Comunidad','asyn_lpa_community']
atributtesNoClustering = nodesNoClustering.loc[:, ['id']+ad_NoClustering+ac_NoClustering]

ad_NoCommunities=['name']
ac_NoCommunities=['Closeness_Centrality','Betweenness_Centrality','Degree_Centrality','Clustering_Coefficient','Triangles','Squares','K_Core']
atributtesNoCommunities = nodesNoCommunities.loc[:, ['id']+ad_NoCommunities+ac_NoCommunities]

ad_NoKCore=['name']
ac_NoKCore=['Closeness_Centrality','Betweenness_Centrality','Degree_Centrality','Clustering_Coefficient','Triangles','Squares','Comunidad','asyn_lpa_community']
atributtesNoKCore = nodesNoKCore.loc[:, ['id']+ad_NoKCore+ac_NoKCore]

#Elegimos el objetivo 
y = nodes['ml_target']


In [124]:
#Manipulación de datos 
#Naive bayes no puede manejar atributos categóricos, por lo que necesitamos convertirlos a numéricos
#Para ello, usamos OrdinalEncoder
codificador_ad= OrdinalEncoder()
codificador_ad.fit(atributtes[ad])
codificador_ad.fit(atributtesNoCentrality[ad_NoCentrality])
codificador_ad.fit(atributtesNoClustering[ad_NoClustering])
codificador_ad.fit(atributtesNoCommunities[ad_NoCommunities])
codificador_ad.fit(atributtesNoKCore[ad_NoKCore])

OrdinalEncoder()

In [125]:
#Transformamos los atributos
atributtes[ad]= codificador_ad.transform(atributtes[ad])
atributtesNoCentrality[ad_NoCentrality]= codificador_ad.transform(atributtesNoCentrality[ad_NoCentrality])
atributtesNoClustering[ad_NoClustering]= codificador_ad.transform(atributtesNoClustering[ad_NoClustering])
atributtesNoCommunities[ad_NoCommunities]= codificador_ad.transform(atributtesNoCommunities[ad_NoCommunities])
atributtesNoKCore[ad_NoKCore]= codificador_ad.transform(atributtesNoKCore[ad_NoKCore])

In [126]:
#Normalizamos name 
scaler = MinMaxScaler(
    feature_range=(0, 1)
)
atributtes['name'] = scaler.fit_transform(atributtes[['name']])
atributtesNoCentrality['name'] = scaler.fit_transform(atributtesNoCentrality[['name']])
atributtesNoClustering['name'] = scaler.fit_transform(atributtesNoClustering[['name']])
atributtesNoCommunities['name'] = scaler.fit_transform(atributtesNoCommunities[['name']])
atributtesNoKCore['name'] = scaler.fit_transform(atributtesNoKCore[['name']])

In [127]:
#Discretizamos los atributos
discretizer = KBinsDiscretizer(
    n_bins=500,
    encode='ordinal',
    strategy='uniform'
)
#Copiamos el dataframe y discretizamos los atributos
atributtes_discretized = atributtes.copy()
atributtes_discretized[ac] = discretizer.fit_transform(atributtes_discretized[ac])

atributtesNoCentrality_discretized = atributtesNoCentrality.copy()
atributtesNoCentrality_discretized[ac_NoCentrality] = discretizer.fit_transform(atributtesNoCentrality_discretized[ac_NoCentrality])

atributtesNoClustering_discretized = atributtesNoClustering.copy()
atributtesNoClustering_discretized[ac_NoClustering] = discretizer.fit_transform(atributtesNoClustering_discretized[ac_NoClustering])

atributtesNoCommunities_discretized = atributtesNoCommunities.copy()
atributtesNoCommunities_discretized[ac_NoCommunities] = discretizer.fit_transform(atributtesNoCommunities_discretized[ac_NoCommunities])

atributtesNoKCore_discretized = atributtesNoKCore.copy()
atributtesNoKCore_discretized[ac_NoKCore] = discretizer.fit_transform(atributtesNoKCore_discretized[ac_NoKCore])


In [128]:
#CON TODOS LOS ATRIBUTOS 

In [129]:
#Validamos el modelo , diviendo los datos en entrenamiento y test
#Alpha hace referencia al suavizado de Laplace 
tub= Pipeline([
    ('preprocess', discretizer),
    ('model', CategoricalNB(alpha=1))
])

#Validación cruzada
cv_results = cross_validate(
    tub,
    atributtes_discretized,
    y,
    cv=10,
    scoring=['recall','accuracy',  'f1_macro']
)
cv_results

{'fit_time': array([0.0616405 , 0.05509138, 0.05919242, 0.05212545, 0.05112624,
        0.06048656, 0.05862308, 0.06939292, 0.06187129, 0.0594666 ]),
 'score_time': array([0.01532555, 0.01219749, 0.00247931, 0.0142014 , 0.0166266 ,
        0.02448177, 0.0019486 , 0.01486301, 0.00806141, 0.01831603]),
 'test_recall': array([0.96402878, 0.69096509, 0.68172485, 0.67453799, 0.69712526,
        0.67556468, 0.72279261, 0.68377823, 0.67145791, 0.96919918]),
 'test_accuracy': array([0.43156499, 0.70557029, 0.71061008, 0.70132626, 0.70795756,
        0.71193634, 0.72785146, 0.71458886, 0.70636605, 0.43899204]),
 'test_f1_macro': array([0.42907444, 0.66485679, 0.66797143, 0.65887014, 0.66778369,
        0.66825524, 0.68876579, 0.67174565, 0.66280745, 0.43684094])}

In [130]:
cv_results['test_accuracy'].mean(), cv_results['test_recall'].mean(), cv_results['test_f1_macro'].mean()

(0.6556763925729443, 0.7431174567532832, 0.6216971563144318)

In [131]:
#CON TODOS LOS ATRIBUTOS EXCEPTO CENTRALITY
##Alpha que hace referencia al suavizado de Laplace = 1
tubNoCentrality= Pipeline([
    ('preprocess', discretizer),
    ('model', CategoricalNB(alpha=1))
])
cv_results_NoCentrality = cross_validate(
    tubNoCentrality,
    atributtesNoCentrality_discretized,
    y,
    cv=10,
    scoring=['recall','accuracy',  'f1_macro']
)
cv_results_NoCentrality

{'fit_time': array([0.04460859, 0.03419185, 0.03139377, 0.0416491 , 0.03668833,
        0.03052998, 0.02994704, 0.03145862, 0.03505349, 0.042629  ]),
 'score_time': array([0.01506352, 0.01881599, 0.02046132, 0.01562285, 0.01720953,
        0.0224576 , 0.01617432, 0.        , 0.01381946, 0.01289582]),
 'test_recall': array([0.98766701, 0.68685832, 0.68480493, 0.68069815, 0.69815195,
        0.68583162, 0.72895277, 0.67043121, 0.66529774, 0.98973306]),
 'test_accuracy': array([0.35384615, 0.72068966, 0.73262599, 0.72281167, 0.72679045,
        0.73209549, 0.74721485, 0.73819629, 0.73474801, 0.36525199]),
 'test_f1_macro': array([0.33773587, 0.67754638, 0.68783895, 0.67854547, 0.68451883,
        0.68751241, 0.70697947, 0.69072602, 0.6868829 , 0.35139722])}

In [132]:
cv_results_NoCentrality['test_accuracy'].mean(), cv_results_NoCentrality['test_recall'].mean(), cv_results_NoCentrality['test_f1_macro'].mean()

(0.6574270557029177, 0.7478426762843171, 0.6189683518552953)

In [133]:
#CON TODOS LOS ATRIBUTOS EXCEPTO CLUSTERING
##Alpha que hace referencia al suavizado de Laplace = 1

tubNoClustering= Pipeline([
    ('preprocess', discretizer),
    ('model', CategoricalNB(alpha=1))
])
cv_results_NoClustering = cross_validate(
    tubNoClustering,
    atributtesNoClustering_discretized,
    y,
    cv=10,
    scoring=['recall','accuracy',  'f1_macro']
)
cv_results_NoClustering

{'fit_time': array([0.09665513, 0.12159681, 0.04628897, 0.06541252, 0.04704905,
        0.04521227, 0.04593468, 0.0610764 , 0.04688883, 0.06070709]),
 'score_time': array([0.02513123, 0.01736474, 0.01514292, 0.00295377, 0.02344394,
        0.01510167, 0.        , 0.01416779, 0.01917219, 0.01303196]),
 'test_recall': array([0.97430627, 0.68480493, 0.68069815, 0.66735113, 0.69712526,
        0.68069815, 0.71868583, 0.67351129, 0.66837782, 0.97535934]),
 'test_accuracy': array([0.40450928, 0.71909814, 0.73395225, 0.71777188, 0.72519894,
        0.72838196, 0.74535809, 0.73076923, 0.72970822, 0.41564987]),
 'test_f1_macro': array([0.39868602, 0.67585417, 0.68843281, 0.67217563, 0.68296502,
        0.68347641, 0.70394608, 0.68455526, 0.68285848, 0.41105492])}

In [134]:
cv_results_NoClustering['test_accuracy'].mean(), cv_results_NoClustering['test_recall'].mean(), cv_results_NoClustering['test_f1_macro'].mean()

(0.6650397877984086, 0.7420918178921222, 0.6284004798358429)

In [135]:
#CON TODOS LOS ATRIBUTOS EXCEPTO COMMUNITIES
##Alpha que hace referencia al suavizado de Laplace = 1

tubNoCommunities= Pipeline([
    ('preprocess', discretizer),
    ('model', CategoricalNB(alpha=1))
])
cv_results_NoCommunities = cross_validate(
    tubNoCommunities,
    atributtesNoCommunities_discretized,
    y,
    cv=10,
    scoring=['recall','accuracy',  'f1_macro']
)
cv_results_NoCommunities

{'fit_time': array([0.05350256, 0.04584646, 0.04745245, 0.03695512, 0.03262067,
        0.06093216, 0.05322766, 0.05498886, 0.04952407, 0.05629516]),
 'score_time': array([0.02037144, 0.01152849, 0.02430654, 0.01671505, 0.01747012,
        0.00318837, 0.00701022, 0.01839471, 0.02466869, 0.0160563 ]),
 'test_recall': array([0.96711202, 0.6036961 , 0.59240246, 0.6026694 , 0.59856263,
        0.57802875, 0.64887064, 0.60061602, 0.57186858, 0.96406571]),
 'test_accuracy': array([0.3530504 , 0.60132626, 0.6132626 , 0.60344828, 0.60106101,
        0.60265252, 0.63448276, 0.61671088, 0.59416446, 0.3596817 ]),
 'test_f1_macro': array([0.3389293 , 0.56488706, 0.57297333, 0.56647331, 0.56393728,
        0.56219828, 0.59854306, 0.57700886, 0.55441566, 0.34716582])}

In [136]:
cv_results_NoCommunities['test_accuracy'].mean(), cv_results_NoCommunities['test_recall'].mean(), cv_results_NoCommunities['test_f1_macro'].mean()

(0.5579840848806367, 0.6727892312140314, 0.5246531962627672)

In [137]:
#CON TODOS LOS ATRIBUTOS EXCEPTO KCORE
##Alpha que hace referencia al suavizado de Laplace = 1

tubNoKCore= Pipeline([
    ('preprocess', discretizer),
    ('model', CategoricalNB(alpha=1))
])
cv_results_NoKCore = cross_validate(
    tubNoKCore,
    atributtesNoKCore_discretized,
    y,
    cv=10,
    scoring=['recall','accuracy',  'f1_macro']
)
cv_results_NoKCore

{'fit_time': array([0.05889106, 0.0691638 , 0.05786443, 0.05472493, 0.05402303,
        0.06275654, 0.06737161, 0.06924701, 0.05973959, 0.04947829]),
 'score_time': array([0.01013613, 0.01576757, 0.01621652, 0.00971031, 0.01668024,
        0.02261972, 0.01902103, 0.01799417, 0.01998091, 0.018857  ]),
 'test_recall': array([0.97738952, 0.67556468, 0.66940452, 0.65195072, 0.6889117 ,
        0.6724846 , 0.71457906, 0.66221766, 0.66221766, 0.97741273]),
 'test_accuracy': array([0.4061008 , 0.71883289, 0.73448276, 0.71856764, 0.7265252 ,
        0.72838196, 0.74535809, 0.73342175, 0.72572944, 0.41220159]),
 'test_f1_macro': array([0.4002931 , 0.67430241, 0.68725685, 0.67057231, 0.68299668,
        0.68228604, 0.70339716, 0.68524022, 0.67842063, 0.40709135])}

In [138]:
cv_results_NoKCore['test_accuracy'].mean(), cv_results_NoKCore['test_recall'].mean(), cv_results_NoKCore['test_f1_macro'].mean()

(0.6649602122015915, 0.7352132843446568, 0.6271856748747873)

In [139]:
#ANALISIS 
print("CON TODOS LOS ATRIBUTOS", cv_results['test_accuracy'].mean())
print("CON TODOS LOS ATRIBUTOS EXCEPTO CENTRALITY", cv_results_NoCentrality['test_accuracy'].mean())
print("CON TODOS LOS ATRIBUTOS EXCEPTO CLUSTERING", cv_results_NoClustering['test_accuracy'].mean())
print("CON TODOS LOS ATRIBUTOS EXCEPTO COMMUNITIES", cv_results_NoCommunities['test_accuracy'].mean())
print("CON TODOS LOS ATRIBUTOS EXCEPTO KCORE", cv_results_NoKCore['test_accuracy'].mean())


CON TODOS LOS ATRIBUTOS 0.6556763925729443
CON TODOS LOS ATRIBUTOS EXCEPTO CENTRALITY 0.6574270557029177
CON TODOS LOS ATRIBUTOS EXCEPTO CLUSTERING 0.6650397877984086
CON TODOS LOS ATRIBUTOS EXCEPTO COMMUNITIES 0.5579840848806367
CON TODOS LOS ATRIBUTOS EXCEPTO KCORE 0.6649602122015915


In [140]:
### Análisis y Conclusiones

#1. **Importancia de las Métricas de Comunidad:**
#   - Al eliminar los atributos relacionados con las comunidades, la precisión del modelo disminuye en un 0.64%, lo que indica que estas métricas juegan un papel crucial en la capacidad del modelo para clasificar correctamente las instancias.

#2. **Impacto de los Atributos de Centralidad:**
#   - Contrario a las expectativas, la eliminación de los atributos de centralidad resulta en una mejora del 0.79% en la precisión del modelo. Esto sugiere que estos atributos pueden no ser tan relevantes para la clasificación en este contexto específico o que podrían estar introduciendo ruido en el modelo.

#3. **Efecto de los Atributos de Agrupamiento y K-Core:**
#   - La exclusión de los atributos de agrupamiento y de k-core tiene un impacto marginal en la precisión del modelo, con cambios menores o incluso una ligera mejora en algunos casos. Esto indica que, aunque útiles, estos atributos no son críticos para lograr una buena precisión en la clasificación.

#En resumen, para nuestro problema específico de clasificación utilizando Naive Bayes, las métricas de comunidad emergen como las más relevantes, mostrando una influencia significativa en la capacidad del modelo para generalizar correctamente. Por otro lado, los atributos de centralidad pueden prescindirse sin comprometer la precisión e incluso mejorándola ligeramente. Los atributos de agrupamiento y k-core, aunque útiles, tienen un impacto menos decisivo en la precisión del modelo.